# Замыкания

Возможность использования вложенными функциями переменных из внешних областей видимости существенно расширяет возможности применения функций. В частности, применение замыканий. В обще смысле замыкание — это функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся ее параметрами. Например, фабрика функций поиска минимума в заранее заданном интервале:

In [1]:
def make_min(*, l, h):
    def inner(a, b, *args):
        args = (a, b) + args
        return min(i for i in args if l <= i <= h)
    return inner

bounded_min = make_min(l=0, h=42)
print(f'{bounded_min(*[-1, 4, 254, 42, 1])}')

1


Здесь при вызове функции ```bounded_min``` фактически происходит вызов ```inner```, находящейся внутри ```make_min```. При этом переменные ```l``` и ```h``` не уничтожаются сборщиком мусора, так как на них все еще остаются ссылки из вложенной функции ```inner```.

Дело в том, что ссылки такие объекты сохраняются в специальном атрибуте объекта функций. Его можно получить используя ```__closure__```. Элементы, хранящиеся в этом кортеже, имеют тип ```cell```. Это специальный тип в Python, предназначенный для внутреннего использования. У элементов кортежа ```__closure__``` есть атрибут ```cell_contents``` с помощью которого можно получить объект.

In [10]:
print(f'{bounded_min.__closure__ = }')
print(f'{type(bounded_min.__closure__[0]) = }')

print(f'{bounded_min.__closure__[0].cell_contents = }')
print(f'{bounded_min.__closure__[1].cell_contents = }')

bounded_min.__closure__ = (<cell at 0x000001A0A2B2EC70: int object at 0x000001A09D046E50>, <cell at 0x000001A0A2B2EA00: int object at 0x000001A09D046910>)
type(bounded_min.__closure__[0]) = <class 'cell'>
bounded_min.__closure__[0].cell_contents = 42
bounded_min.__closure__[1].cell_contents = 0


Выполните этот код в [pythontutor](http://www.pythontutor.com/visualize.html#code=def%20make_min%28*,%20l,%20h%29%3A%0A%20%20%20%20def%20inner%28a,%20b,%20*args%29%3A%0A%20%20%20%20%20%20%20%20args%20%3D%20%28a,%20b%29%20%2B%20args%0A%20%20%20%20%20%20%20%20return%20min%28i%20for%20i%20in%20args%20if%20l%20%3C%3D%20i%20%3C%3D%20h%29%0A%20%20%20%20return%20inner%0A%0Abounded_min%20%3D%20make_min%28l%3D0,%20h%3D42%29%0Ares%20%3D%20bounded_min%28*%5B-1,%204,%20254,%2042,%201%5D%29%0Aprint%28res%29&cumulative=false&curInstr=0&heapPrimitives=true&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false) и посмотрите, что происходит в памяти в момент выполнения.

Переменные можно использовать не только из объемлющей области видимости.

In [12]:
def foo():  # плохая практика
    print(i)

for i in range(4):
    foo()

0
1
2
3


## Позднее связывание

Рассмотрим случай, когда необходимо создать несколько функций.

In [14]:
def create_multipiers():
    return [lambda x: i * x for i in range(3)]

for multiplier in create_multipiers():
    print(multiplier(5))

10
10
10


Выполните этот код в [pythontutor](http://www.pythontutor.com/visualize.html#code=def%20create_multipiers%28%29%3A%0A%20%20%20%20return%20%5Blambda%20x%3A%20i%20*%20x%20for%20i%20in%20range%283%29%5D%0A%0Afor%20multiplier%20in%20create_multipiers%28%29%3A%0A%20%20%20%20print%28multiplier%285%29%29&cumulative=false&curInstr=0&heapPrimitives=true&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false) и посмотрите, что происходит в памяти в момент выполнения.

Результат отличается от ожидаемого. Это происходит из-за поздней привязки в Python, которое заключается в том, что поиск значений переменных, используемых в замыканиях, происходит во время вызова внутренней функции. Таким образом, в приведенном выше коде всякий раз, когда вызывается какая-либо из возвращаемых функций (т.е. ```multiplier```), значение ```i``` ищется в окружающей области видимости во время ее вызова (а к тому времени цикл уже завершился, поэтому ```i``` уже был присвоен конечный результат — значение ```2```).

Для решения этой проблемы достаточно явно передать в каждое замыкание (т.е. лямбду) переменную ```i``` используя именованные аргументы.

In [15]:
def create_multipiers():
    return [lambda x, i=i: i * x for i in range(3)]

for multiplier in create_multipiers():
    print(multiplier(5))

0
5
10


# Полезные ссылки

- [Python Inner Functions—What Are They Good For?](https://realpython.com/inner-functions-what-are-they-good-for/)
- [Why aren't python nested functions called closures?](https://stackoverflow.com/questions/4020419/why-arent-python-nested-functions-called-closures)